In [ ]:
from flask import Flask, jsonify, request

# Library used for opening and reading
import urllib.request

# Used for parsing all the tables present in the WebSite
#from html_table_parser.parser import HTMLTableParser

# Used to Convert Parsed data into a pandas dataframe
import pandas as pd

# Importing Date for creating file :
from datetime import datetime

# Importing Json
import json

# from pymongo import MongoClient
import pymongo

import traceback

# Creating Database connection
myclient = pymongo.MongoClient("localhost", 27017)

# database
db = myclient["Cars_test"]

# Created or Switched to collection
Collection = db["Data_test"]



In [ ]:
# http://127.0.0.1:5000/generate_json?url=https://www.auto-data.net/en/audi-100-avant-4a-c4-2.8-e-174hp-automatic-26292
# creating a Flask app
app = Flask(__name__)
@app.route("/generate_json")
def generate_json():

    try:
        url = request.args.get("url")
        # defining the html contents of a URL.
        xhtml = get_data(url).decode("utf-8")

        # Defining the HTMLTableParser object
        parser = HTMLTableParser()

        # Storing the returned html data in the HTMLTableParser object.
        parser.feed(xhtml)

        # Converting the parsed data to PandasDataFrame.
        pd.set_option("display.max_rows", len(parser.tables[1]))
        print("\n\nPANDAS DATAFRAME\n")
        print(pd.DataFrame(parser.tables[1]))
        df = pd.DataFrame(parser.tables[1])
        df.dropna(inplace=True)

        # now = datetime.now()
        # dt_string = now.strftime("%d%m%Y%H%M%S")
        df.to_json("temp" ".json",  orient = 'values')

        dump_data()

        return jsonify("Data Inserted Successfully")
    except Exception as ex:
        print("Exception in generate_json : ",(str(ex)))
        print(traceback.print_exc())
        return jsonify("Exception in generate_json.")

In [ ]:
# This functions opens a website and reads its contents (HTTP Response Body)
def get_data(url):
    try:
        # making request to the website
        req = urllib.request.Request(url=url)
        d = urllib.request.urlopen(req)

        # reading contents of the website
        return d.read()
    except Exception as ex:
        print("Exception in get_data : ",(str(ex)))

In [ ]:
# Creating UDF for importing Data into mongoDB
def dump_data():
    try:
        # Loading or Opening the json file
        strData = ""
        with open("temp.json") as reader:
            for i in reader:
                strData = i
            strData = {strData: strData}
            print("Data Type : ", type(strData))
        # file_data = json.load(file)

        # Inserting the loaded data in the Collection
        Collection.insert_many([strData])
    except Exception as ex:
        print("Exception in dump : ",(str(ex)))

In [ ]:
# driver function
if __name__ == "__main__":
    app.run(debug=True)